In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
doc = []
with open("data/dates.txt") as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [3]:
def matcher(x):
    # For later use
    month_mapper = {
        1 : ("Jan", "Jan.", "January", "January."),
        2 : ("Feb", "Feb.", "February", "February."),
        3 : ("Mar", "Mar.", "March", "March."),
        4 : ("Apr", "Apr.", "April", "April."),
        5 : ("May", "May.", "May", "May."),
        6 : ("Jun", "Jun.", "June", "June."),
        7 : ("Jul", "Jul.", "July", "July."),
        8 : ("Aug", "Aug.", "August", "August."),
        9 : ("Sep", "Sep.", "September", "September."),
        10 : ("Oct", "Oct.", "October", "October."),
        11 : ("Nov", "Nov.", "November", "November."),
        12 : ("Dec", "Dec.", "December", "December."),
    }
    ############################## MATCHER 1 ##############################
    try:
        ###### MATCH: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        match = re.findall(r"[\d]{1,2}[/\-][\d]{1,2}[/\-][\d]{2,4}", x)[0]
        ###### PROCESSING
        if "/" in match:
            items = match.split("/")
        if "-" in match:
            items = match.split("-")

        if len(items[0]) == 1:
            items[0] = "0" + items[0]

        if len(items[1]) == 1:
            items[1] = "0" + items[1]

        if len(items[2]) == 2:
            items[2] = "19" + items[-1]

        processed_match = "/".join(items)
        return processed_match
    except:        
        ############################## MATCHER 2 ##############################
        try:
            ###### MATCH: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
            match = re.findall(r"[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{1,2}[\-\s\,]{1,2}[\d]{4}", x)[0]
            ###### PROCESSING
            items = match.replace(",", "").split(" ")

            for i, k in month_mapper.items():
                if items[0] in k:
                    items[0] = str(i)

            if len(items[0]) == 1:
                items[0] = "0" + items[0]

            if len(items[1]) == 1:
                items[1] = "0" + items[1]

            processed_match = "/".join(items)
            return processed_match
        except:
            ############################## MATCHER 3 ##############################
            try:
                ###### MATCH: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
                match = re.findall(r"[\d]{1,2}\s[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{4}", x)[0]
                ###### PROCESSING
                items = match.replace(",", "").split(" ")

                for i, k in month_mapper.items():
                    if items[1] in k:
                        items[1] = str(i)

                if len(items[0]) == 1:
                    items[0] = "0" + items[0]

                if len(items[1]) == 1:
                    items[1] = "0" + items[1]

                day = items[0]
                month = items[1]

                items[1] = day
                items[0] = month

                processed_match = "/".join(items)
                return processed_match
            except:
                ############################## MATCHER 4 ##############################
                try:
                    ###### MATCH: Feb 2009; Sep 2009; Oct 2010
                    match = re.findall(r"[^\W\d_]{3,}\s[\d]{4}", x)[0]
                    ###### PROCESSING
                    items = match.replace(",", "").split(" ")

                    for i, k in month_mapper.items():
                        if items[0] in k:
                            items[0] = str(i)

                    if len(items[0]) == 1:
                        items[0] = "0" + items[0]

                    items.insert(1, "01")

                    processed_match = "/".join(items)
                    return processed_match
                except:
                    ############################## MATCHER 5 ##############################
                    try:
                        ###### MATCH: 6/2008; 12/2009
                        match = re.findall(r"[\d]{1,2}/[\d]{4}", x)[0]
                        ###### PROCESSING
                        items = match.replace(",", "").split("/")

                        if len(items[0]) == 1:
                            items[0] = "0" + items[0]

                        items.insert(1, "01")

                        processed_match = "/".join(items)
                        return processed_match
                    except:
                        ############################## MATCHER 6 ##############################
                        try:
                            ###### MATCH: 2009; 2010
                            match = re.findall(r"[\d]{4}", x)[0]
                            ###### PROCESSING
                            items = match.replace(",", "").split("/")

                            items.insert(0, "01")
                            items.insert(0, "01")

                            processed_match = "/".join(items)
                            return processed_match
                        except:
                            return np.nan

######################################################################
test = df.apply(lambda x: matcher(x))

print(f"Solved cases: {test.notna().sum()}")
test[test.notna()]

Solved cases: 500


0      03/25/1993
1      06/18/1985
2      07/08/1971
3      09/27/1975
4      02/06/1996
          ...    
495    01/01/1979
496    01/01/2006
497    01/01/2008
498    01/01/2005
499    01/01/1980
Length: 500, dtype: object

In [4]:
# Outliers
outliers = [235, 245, 256, 281, 283, 285, 298, 313, 333, 459, 460, 461, 467, 468, 479, 487, 488]

for outlier in outliers:
    print(f"Outlier row = {outlier}")
    print(df[outlier])
    print(test[outlier])
    print("-" * 50)

Outlier row = 235
pOct 2015 - Admitted to Gray Clinic for depression, alcohol intox/dependence and SI.  Discharged to MH Partial Hospital---on trazadone 50 mg QHS, Ativan 0.5 mg BID, Wellbutrin 150 mg BID, Seroquel 50 mg QHS and Zoloft 100 mg.  It is unclear if patient attended Memorial partial hospital program.  By report she had refills of trazadone and sertraline via PCP (see HPI).Hx of Outpatient Treatment: Yes

pOct/01/2015
--------------------------------------------------
Outlier row = 245
lNovember 1990 - NPCCHx of Outpatient Treatment: Yes

lNovember/01/1990
--------------------------------------------------
Outlier row = 256
IAug 1988: Symmes Hospital s/p SA OD with ASA

IAug/01/1988
--------------------------------------------------
Outlier row = 281
yAug 2004 - Dr. Tom Ngo at TMC; dx GAD; patient stopped after two visits re: "I was scared if this meant I was sick and really needed this", despite success in controlling anxiety-related symptoms of nausea after just one treatm

In [5]:
# 235, 245
from datetime import datetime

s = "05/01/2005"
datetime.strptime(s, "%m/%d/%Y")

datetime.datetime(2005, 5, 1, 0, 0)

In [6]:
def to_time(x):
    try:
        return datetime.strptime(x, "%m/%d/%Y")
    except:
        return np.nan

In [7]:
test.apply(lambda x: to_time(x)).sort_values().isna().sum()

17

# Putting everything together

In [42]:
def date_sorter():
    import re
    import numpy as np
    from datetime import datetime

    ######################################## GET DATES ########################################
    def matcher(x):
        # For later use
        month_mapper = {
            1 : ("Jan", "Jan.", "January", "January."),
            2 : ("Feb", "Feb.", "February", "February."),
            3 : ("Mar", "Mar.", "March", "March."),
            4 : ("Apr", "Apr.", "April", "April."),
            5 : ("May", "May.", "May", "May."),
            6 : ("Jun", "Jun.", "June", "June."),
            7 : ("Jul", "Jul.", "July", "July."),
            8 : ("Aug", "Aug.", "August", "August."),
            9 : ("Sep", "Sep.", "September", "September."),
            10 : ("Oct", "Oct.", "October", "October."),
            11 : ("Nov", "Nov.", "November", "November."),
            12 : ("Dec", "Dec.", "December", "December."),
        }
        ############################## MATCHER 1 ##############################
        try:
            ###### MATCH: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
            match = re.findall(r"[\d]{1,2}[/\-][\d]{1,2}[/\-][\d]{2,4}", x)[0]
            ###### PROCESSING
            if "/" in match:
                items = match.split("/")
            if "-" in match:
                items = match.split("-")

            if len(items[0]) == 1:
                items[0] = "0" + items[0]

            if len(items[1]) == 1:
                items[1] = "0" + items[1]

            if len(items[2]) == 2:
                items[2] = "19" + items[-1]

            processed_match = "/".join(items)
            return processed_match
        except:        
            ############################## MATCHER 2 ##############################
            try:
                ###### MATCH: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
                match = re.findall(r"[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{1,2}[\-\s\,]{1,2}[\d]{4}", x)[0]
                ###### PROCESSING
                items = match.replace(",", "").split(" ")

                for i, k in month_mapper.items():
                    if items[0] in k:
                        items[0] = str(i)

                if len(items[0]) == 1:
                    items[0] = "0" + items[0]

                if len(items[1]) == 1:
                    items[1] = "0" + items[1]

                processed_match = "/".join(items)
                return processed_match
            except:
                ############################## MATCHER 3 ##############################
                try:
                    ###### MATCH: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
                    match = re.findall(r"[\d]{1,2}\s[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{4}", x)[0]
                    ###### PROCESSING
                    items = match.replace(",", "").split(" ")

                    for i, k in month_mapper.items():
                        if items[1] in k:
                            items[1] = str(i)

                    if len(items[0]) == 1:
                        items[0] = "0" + items[0]

                    if len(items[1]) == 1:
                        items[1] = "0" + items[1]

                    day = items[0]
                    month = items[1]

                    items[1] = day
                    items[0] = month

                    processed_match = "/".join(items)
                    return processed_match
                except:
                    ############################## MATCHER 4 ##############################
                    try:
                        ###### MATCH: Feb 2009; Sep 2009; Oct 2010
                        match = re.findall(r"[^\W\d_]{3,}\s[\d]{4}", x)[0]
                        ###### PROCESSING
                        items = match.replace(",", "").split(" ")

                        for i, k in month_mapper.items():
                            if items[0] in k:
                                items[0] = str(i)

                        if len(items[0]) == 1:
                            items[0] = "0" + items[0]

                        items.insert(1, "01")

                        processed_match = "/".join(items)
                        return processed_match
                    except:
                        ############################## MATCHER 5 ##############################
                        try:
                            ###### MATCH: 6/2008; 12/2009
                            match = re.findall(r"[\d]{1,2}/[\d]{4}", x)[0]
                            ###### PROCESSING
                            items = match.replace(",", "").split("/")

                            if len(items[0]) == 1:
                                items[0] = "0" + items[0]

                            items.insert(1, "01")

                            processed_match = "/".join(items)
                            return processed_match
                        except:
                            ############################## MATCHER 6 ##############################
                            try:
                                ###### MATCH: 2009; 2010
                                match = re.findall(r"[\d]{4}", x)[0]
                                ###### PROCESSING
                                items = match.replace(",", "").split("/")

                                items.insert(0, "01")
                                items.insert(0, "01")

                                processed_match = "/".join(items)
                                return processed_match
                            except:
                                return np.nan

    ######################################## PROCESSING ########################################
    def to_time(x):
        try:
            return datetime.strptime(x, "%m/%d/%Y")
        except:
            return np.nan

    # 1) GET DATES
    df2 = df.apply(lambda x: matcher(x))

    # 2) CLEAN OUTLIERS
    df2[235] = "10/01/2015"
    df2[245] = "11/01/1990"
    df2[256] = "08/01/1988"
    df2[281] = "08/01/2004"
    df2[283] = "02/01/1977"
    df2[285] = "09/01/1977"
    df2[298] = "01/01/1993"
    df2[313] = "12/01/1978"
    df2[333] = "11/01/1997"
    df2[459] = "01/01/1998"
    df2[460] = "01/01/2012"
    df2[461] = "01/01/1991"
    df2[467] = "01/01/2011"
    df2[468] = "01/01/1997"
    df2[479] = "01/01/1996"
    df2[487] = "01/01/1992"
    df2[488] = "01/01/1977"

    # 3) TRANSFORM INTO DATE FORMAT
    df2 = df2.apply(lambda x: to_time(x))

    # 4) SORTING DATA
    df2 = df2.reset_index()
    # ordered_df = df2.copy()

    # ordered_df = ordered_df.sort_values(by = 0)
    # ordered_df.insert(0, "position", range(len(ordered_df)))
    # full_df = pd.merge(df2, ordered_df, how = "inner", on = "index")
    # full_df["text"] = df

    # return full_df.position
    #return df2.sort_values(by = 0).index
    df2["text"] = df
    return df2.sort_values(by = 0)

In [53]:
date_sorter()[200:250]

,index,0,text
63,63,1987-06-28,6/28/87 Impression Strengths/Abilities:\n
416,416,1987-07-01,s1 CSU stay at Donelson Hospital 7/1987 in the...
55,55,1987-10-11,10/11/1987 CPT Code: 90791: No medical services\n
130,130,1987-10-25,25 Oct 1987 Total time of visit (in minutes):\n
116,116,1987-12-04,12/04/87 SOS-10 Total Score:\n
76,76,1987-12-05,12/5/87 Total time of visit (in minutes):\n
462,462,1988-01-01,sLexapro (1988-now): Good response (anxiety)\n
330,330,1988-04-01,"""-worked with Uma Dewitt (ED specialist) x 10 ..."
37,37,1988-05-24,5/24/88 CPT Code: 90792: With medical services\n
390,390,1988-08-01,08/1988 Primary Care Doctor:\n


In [55]:
date_sorter().loc[51, "text"]

'"In the context of the patient\'s work up, she had Neuropsychological  testing 5/24/91.  She was referred because of her cognitive decline and symptoms of motor dysfunction to help characterize her current functioning. :\n'

In [12]:
from functools import reduce

def test_():
    # Your code here
    mon2int = dict([(m, i+1) for i, m in enumerate('jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec'.split(","))])
    rule1 = r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}|\d{4})'
    rule2 = r'(?P<month>(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\w*)[.]?[ -](?P<day>\d{1,2})(?:\w{2})?[,]?[ -](?P<year>\d{4})'
    rule3 = r'(?P<day>\d{1,2}) (?P<month>(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\w*)[.,]? (?P<year>\d{4})'
    rule4 = rule2
    rule5 = r'(?P<day>)(?P<month>(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\w*) (?P<year>\d{4})'
    rule6 = r'(?P<day>)(?P<month>\d{1,2})/(?P<year>\d{4})'
    rule7 = r'(?P<day>)(?P<month>)(?:\D|^)(?P<year>\d{4})(?:\D|$)'
    rule_list = [rule1, rule2, rule3, rule4, rule5, rule6, rule7]
    extract_df_list = [df.str.lower().str.extract(rule) for rule in rule_list]
    date_df = reduce(lambda x,y: x.fillna(y),extract_df_list)
    date_df['text'] = df
    date_df['month'] = date_df['month'].apply(lambda x: mon2int.get(str(x)[:3],x))
    date_df['month'] = date_df['month'].fillna('1').apply(lambda x: '1' if x=='' else x)
    date_df['year'] = date_df['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    date_df['day'] = date_df['day'].fillna('1').apply(lambda x: '1' if x=='' else x)
    date_df = date_df[["year","month","day"]].astype(int)
    index_df = date_df.sort_values(['year','month','day'], axis=0, ascending=False).reset_index()
    index_df = index_df['index']
    return index_df

test()

0      413
1      161
2      186
3      141
4      231
      ... 
495     14
496     95
497     30
498     45
499     77
Name: index, Length: 500, dtype: int64